<a href="https://colab.research.google.com/github/WaHMugdha/MoodBites/blob/main/moodbites_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pickle
# Load the pickle file for word2vec model
with open('/content/drive/MyDrive/Kaggle/word2vec_model.pkl', 'rb') as f:
    word2vec_model = pickle.load(f)

# Load the pickle file for dataset
with open('/content/drive/MyDrive/Kaggle/data_after_word2vec.pkl', 'rb') as f:
    data = pickle.load(f)

data = data.drop(columns=['Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'])
data['cleaned_reviews'] = data['cleaned_reviews'].apply(lambda tokens: ' '.join(tokens))
data.head()

Mounted at /content/drive


,url,name,online_order,book_table,rate,votes,location,rest_type,cuisines,cost,reviews_list,type,city,Sentiment_Score,Sentiment_Class,modified_reviews_list,cleaned_reviews
0,https://www.zomato.com/bangalore/jalsa-banasha...,2933,1,1,0.720882,0.611201,1,24,1972,0.557298,"('Rated 4.0', 'RATED\n A beautiful place to d...",0,1,0.7430,3,A beautiful place to dine in.The interiors tak...,beautiful place dine interior take back mughal...
1,https://www.zomato.com/bangalore/jalsa-banasha...,2933,1,1,0.720882,0.611201,1,24,1972,0.557298,"('Rated 4.0', 'RATED\n I was here for dinner ...",0,1,0.9486,4,I was here for dinner with my family on a week...,dinner family weekday restaurant completely em...
2,https://www.zomato.com/bangalore/jalsa-banasha...,2933,1,1,0.720882,0.611201,1,24,1972,0.557298,"('Rated 2.0', 'RATED\n Its a restaurant near ...",0,1,0.7964,3,Its a restaurant near to Banashankari BDA. Me ...,restaurant near banashankari bda along office ...
3,https://www.zomato.com/bangalore/jalsa-banasha...,2933,1,1,0.720882,0.611201,1,24,1972,0.557298,"('Rated 4.0', 'RATED\n We went here on a week...",0,1,0.9622,4,We went here on a weekend and one of us had th...,go weekend one u buffet two u take ala carte f...
4,https://www.zomato.com/bangalore/jalsa-banasha...,2933,1,1,0.720882,0.611201,1,24,1972,0.557298,"('Rated 5.0', 'RATED\n The best thing about t...",0,1,0.9461,4,The best thing about the place is itÃ\x83\x83Ã...,best thing place ambiance second best thing yu...


In [ ]:
data.columns

Index(['url', 'name', 'online_order', 'book_table', 'rate', 'votes',
       'location', 'rest_type', 'cuisines', 'cost', 'reviews_list', 'type',
       'city', 'Sentiment_Score', 'Sentiment_Class', 'modified_reviews_list',
       'cleaned_reviews'],
      dtype='object')

In [ ]:
#adding the word embedding to the dataset and removing unnecessary columns
import numpy as np
from sklearn.preprocessing import StandardScaler

data.drop(columns=['votes', 'reviews_list', 'modified_reviews_list'], inplace=True)
data.rename(columns={'type': 'dining_type','cost': 'cost(for 2)'}, inplace=True)

scaler = StandardScaler()
columns_to_scale = ['cuisines', 'rest_type']
data[columns_to_scale] = scaler.fit_transform(data[columns_to_scale])
import gensim
def text_to_word_embeddings(text, word2vec_model):
    word_embeddings = []
    for word in text.split():
        if word in word2vec_model.wv:
            word_embeddings.append(word2vec_model.wv[word])
    return np.array(word_embeddings)

data['word_embeddings_of_reviews'] = data['cleaned_reviews'].apply(text_to_word_embeddings, args=(word2vec_model,))
data.head()

,url,name,online_order,book_table,rate,location,rest_type,cuisines,cost(for 2),dining_type,city,Sentiment_Score,Sentiment_Class,cleaned_reviews,word_embeddings_of_reviews
0,https://www.zomato.com/bangalore/jalsa-banasha...,2933,1,1,0.720882,1,-0.684284,1.1181,0.557298,0,1,0.7430,3,beautiful place dine interior take back mughal...,"[[2.0981128, 1.815063, -6.8484993, 1.6597033, ..."
1,https://www.zomato.com/bangalore/jalsa-banasha...,2933,1,1,0.720882,1,-0.684284,1.1181,0.557298,0,1,0.9486,4,dinner family weekday restaurant completely em...,"[[0.77261764, 3.2976336, 1.0144088, -5.099168,..."
2,https://www.zomato.com/bangalore/jalsa-banasha...,2933,1,1,0.720882,1,-0.684284,1.1181,0.557298,0,1,0.7964,3,restaurant near banashankari bda along office ...,"[[4.081995, 4.763423, -2.1862926, 3.2217922, -..."
3,https://www.zomato.com/bangalore/jalsa-banasha...,2933,1,1,0.720882,1,-0.684284,1.1181,0.557298,0,1,0.9622,4,go weekend one u buffet two u take ala carte f...,"[[2.0472045, 0.7242372, 3.7346416, -0.51295847..."
4,https://www.zomato.com/bangalore/jalsa-banasha...,2933,1,1,0.720882,1,-0.684284,1.1181,0.557298,0,1,0.9461,4,best thing place ambiance second best thing yu...,"[[1.6863357, 7.823443, -0.32242757, -0.3208254..."


In [ ]:
data['location'].nunique()

90

In [ ]:
# location = 22 is the BTM neighbourhood  Byrasandra, Tavarekere and Madiwala Layout
data['location'].value_counts().sort_values(ascending=False)

44    132029
0      84821
27     73570
22     64408
55     56257
       ...  
81        29
38        27
23        17
88        15
65         9
Name: location, Length: 90, dtype: int64

In [ ]:
new_df = data[data['location'] == 22]

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
# columns_to_scale = ['dining_type', 'city']
new_df[['dining_type']] = scaler.fit_transform(new_df[['dining_type']])
new_df.head()

In [ ]:
new_df.shape

(64408, 15)

In [ ]:
import pickle
with open('/content/drive/MyDrive/Kaggle/data location = 22(BTM).pkl', 'wb') as f:
  pickle.dump(new_df, f)

---
---



In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
data_btm = pd.read_pickle('/content/drive/MyDrive/Kaggle/data location = 22(BTM).pkl')
data_btm.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,url,name,online_order,book_table,rate,location,rest_type,cuisines,cost(for 2),dining_type,city,Sentiment_Score,Sentiment_Class,cleaned_reviews,word_embeddings_of_reviews
54161,https://www.zomato.com/bangalore/navya-foods-h...,4124,1,0,0.286423,22,1.278466,1.001651,-0.582052,-0.486294,3,0.9002,4,boneless biriyani good like much homely foods ...,"[[0.72709286, 2.2768533, 3.0795612, -5.3143263..."
69129,https://www.zomato.com/bangalore/the-paratha-c...,6375,1,0,0.845013,22,-0.203611,0.647824,-0.354182,-0.486294,4,0.7885,3,x ordered place multiple time love food favour...,"[[3.6359832, -0.41793633, 2.6480174, -0.356185..."
69130,https://www.zomato.com/bangalore/the-paratha-c...,6375,1,0,0.845013,22,-0.203611,0.647824,-0.354182,-0.486294,4,0.9623,4,new outlet hsr serve best food hsr guess try a...,"[[2.5347857, 0.22424214, 1.9644408, 3.3490353,..."
69131,https://www.zomato.com/bangalore/the-paratha-c...,6375,1,0,0.845013,22,-0.203611,0.647824,-0.354182,-0.486294,4,0.8297,4,place order aloo paratha dal makhani box parat...,"[[3.2815177, 2.9132953, -1.1734662, 2.2773705,..."
69132,https://www.zomato.com/bangalore/the-paratha-c...,6375,1,0,0.845013,22,-0.203611,0.647824,-0.354182,-0.486294,4,0.9854,4,go zomato find place place order aloo paratha ...,"[[2.0472045, 0.7242372, 3.7346416, -0.51295847..."


In [ ]:
data_btm.shape

(64408, 15)

# SVM
---
(WITHOUT WORD EMBEDDINGS)



In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, recall_score
from sklearn.svm import SVC
data_btm['Sentiment_Class'] = data_btm['Sentiment_Class'].astype(int)

x = data_btm[['online_order', 'book_table', 'rate', 'rest_type', 'cuisines', 'cost(for 2)', 'dining_type' ]]
y = data_btm['Sentiment_Class']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
x_train.shape

(51526, 7)

In [ ]:
data_btm['Sentiment_Class'].value_counts()

4    31409
3    15565
2    10030
1     6019
0     1385
Name: Sentiment_Class, dtype: int64

In [ ]:
svm_model = SVC(kernel='poly', C=1.0, decision_function_shape='ovr')
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
svm_model.fit(x_train, y_train)

SVC(kernel='poly')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

y_pred = svm_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.5424623505666822
Precision: 0.4549462623328751
Recall: 0.5424623505666822
F1-score: 0.4560381105809704
Confusion Matrix:
 [[   0    1   69    9  196]
 [   0    6  415   90  656]
 [   0    8 1006  218  811]
 [   0   11  606  259 2198]
 [   0   11  438  157 5717]]


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


1. **Accuracy**: The model's predictions are correct for about 54.2% of the instances.

2. **Precision**: The model predicts a positive instance, it is correct about 45.5% of the time.

3. **Recall**: The model correctly identifies about 54.2% of the actual positive instances.

4. **F1-score**: The F1-score is approximately 0.456, reflecting the balance between precision and recall in the model's predictions.

5. **Confusion Matrix**:
  - 0 instances were correctly classified as Class 1 (True Negatives).
  - 1 instance was incorrectly classified as Class 2 (False Positives).
  - 6 instances were correctly classified as Class 2 (True Positives).
  - 1006 instances were correctly classified as Class 3 (True Positives).
  - 259 instances were correctly classified as Class 4 (True Positives).
  - 5717 instances were correctly classified as Class 5 (True Positives).
  - rest were incorrectly classified

{Class 1: very bad, Class 2: bad, Class 3: neutral, Class 4: good, Class 5: very good}




In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       275
           1       0.16      0.01      0.01      1167
           2       0.40      0.49      0.44      2043
           3       0.35      0.08      0.14      3074
           4       0.60      0.90      0.72      6323

    accuracy                           0.54     12882
   macro avg       0.30      0.30      0.26     12882
weighted avg       0.45      0.54      0.46     12882



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
import pickle
file_path = '/content/drive/MyDrive/Kaggle/svm_model_model.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(svm_model, file)

# SVM
---
(WITH WORD EMBEDDINGS)


In [ ]:
import numpy as np
def compute_awe(embeddings_sequences):
    awe_list = [] #empty list to store average embeddings
    # Iterate over each sequence of word embeddings
    for sequence in embeddings_sequences:
        # Compute the average embedding for the sequence
        average_embedding = np.mean(sequence, axis=0)
        awe_list.append(average_embedding)
    return np.array(awe_list)

In [ ]:
from sklearn.model_selection import train_test_split
y = data_btm['Sentiment_Class']

awe_features = compute_awe(data_btm['word_embeddings_of_reviews'])
x_other_features = data_btm[['online_order', 'book_table', 'rate', 'rest_type', 'cuisines', 'cost(for 2)', 'dining_type']]
x = np.concatenate([x_other_features, awe_features], axis=1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
svm_model = SVC(kernel='poly', C=1.0, decision_function_shape='ovr')
params_grid = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
svm_model.fit(x_train, y_train)

SVC(kernel='poly')

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

y_pred = svm_model.predict(x_test)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)
print("Confusion Matrix:\n", conf_matrix)

Accuracy: 0.8539823008849557
Precision: 0.8593077130512863
Recall: 0.8539823008849557
F1-score: 0.8499338421174509
Confusion Matrix:
 [[ 174   47    3   23   28]
 [   8  924   18  100  117]
 [  14   45 1653  110  221]
 [   4   45   13 2096  916]
 [   0    4    2  163 6154]]


In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.63      0.73       275
           1       0.87      0.79      0.83      1167
           2       0.98      0.81      0.89      2043
           3       0.84      0.68      0.75      3074
           4       0.83      0.97      0.89      6323

    accuracy                           0.85     12882
   macro avg       0.88      0.78      0.82     12882
weighted avg       0.86      0.85      0.85     12882



In [ ]:
import pickle
file_path = '/content/drive/MyDrive/Kaggle/svm_model_with_word_embeddings.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(svm_model, file)

# NEURAL NETWORKS
---
RNN, LSTM models


In [ ]:
numeric_columns = data_btm.select_dtypes(include=['float64', 'int64']).columns
data_btm[numeric_columns] = data_btm[numeric_columns].astype('float32')

In [ ]:
data_btm.dtypes

url                            object
name                          float32
online_order                  float32
book_table                    float32
rate                          float32
location                      float32
rest_type                     float32
cuisines                      float32
cost(for 2)                   float32
dining_type                   float32
city                          float32
Sentiment_Score               float32
Sentiment_Class               float32
cleaned_reviews                object
word_embeddings_of_reviews     object
dtype: object

In [ ]:
with open('/content/drive/MyDrive/Kaggle/word2vec_model.pkl', 'rb') as f:
    word2vec_model = pickle.load(f)

In [ ]:
vocab_size = len(word2vec_model.wv.key_to_index)
print("Vocabulary Size:", vocab_size)

Vocabulary Size: 56803


In [ ]:
import numpy as np
def compute_awe(embeddings_sequences):
    awe_list = []
    for sequence in embeddings_sequences:
        average_embedding = np.mean(sequence, axis=0)
        awe_list.append(average_embedding)
    return np.array(awe_list)

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
y = data_btm['Sentiment_Class']
awe_features = compute_awe(data_btm['word_embeddings_of_reviews'])
x_other_features = data_btm[['online_order', 'book_table', 'rate', 'rest_type', 'cuisines', 'cost(for 2)', 'dining_type']]
x = np.concatenate([x_other_features, awe_features], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
print(x_train.shape, y_train.shape)

(51526, 39) (51526,)


## Simple RNN
---

In [ ]:
from tensorflow.keras import layers, Sequential

model = Sequential([
    layers.Embedding(vocab_size, 128),  # Embedding layer for word embeddings
    layers.SimpleRNN(128),  # Simple RNN layer
    layers.Dense(64, activation='relu'),  # Dense layer with ReLU activation
    layers.Dense(5, activation='softmax')  # Output layer with softmax activation
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
y_train_categorical = tf.keras.utils.to_categorical(y_train, num_classes=5)
y_test_categorical = tf.keras.utils.to_categorical(y_test, num_classes=5)
model.fit(x_train, y_train_categorical, epochs=10, batch_size=32)
loss, accuracy = model.evaluate(x_test, y_test_categorical)

print("Loss:", loss)
print("Accuracy:", accuracy)

Epoch 1/10
1611/1611 [==============================] - 73s 43ms/step - loss: 0.8575 - accuracy: 0.6569
Epoch 2/10
1611/1611 [==============================] - 71s 44ms/step - loss: 0.6254 - accuracy: 0.7631
Epoch 3/10
1611/1611 [==============================] - 67s 42ms/step - loss: 0.5022 - accuracy: 0.8189
Epoch 4/10
1611/1611 [==============================] - 67s 42ms/step - loss: 0.4332 - accuracy: 0.8485
Epoch 5/10
1611/1611 [==============================] - 66s 41ms/step - loss: 0.3960 - accuracy: 0.8621
Epoch 6/10
1611/1611 [==============================] - 65s 40ms/step - loss: 0.3661 - accuracy: 0.8736
Epoch 7/10
1611/1611 [==============================] - 65s 41ms/step - loss: 0.3640 - accuracy: 0.8746
Epoch 8/10
1611/1611 [==============================] - 64s 40ms/step - loss: 0.3440 - accuracy: 0.8825
Epoch 9/10
1611/1611 [==============================] - 76s 47ms/step - loss: 0.3490 - accuracy: 0.8800
Epoch 10/10
403/403 [==============================] - 2s 5ms/st

In [ ]:
from sklearn.metrics import classification_report
y_pred_prob = model.predict(x_test)
y_pred = np.argmax(y_pred_prob, axis=1)
print(classification_report(y_test, y_pred))

403/403 [==============================] - 2s 5ms/step
              precision    recall  f1-score   support

           0       0.82      0.69      0.75       275
           1       0.83      0.83      0.83      1167
           2       0.95      0.85      0.90      2043
           3       0.83      0.81      0.82      3074
           4       0.91      0.95      0.93      6323

    accuracy                           0.88     12882
   macro avg       0.87      0.83      0.84     12882
weighted avg       0.88      0.88      0.88     12882



In [ ]:
import pickle
file_path = '/content/drive/MyDrive/Kaggle/simpleRNN_model.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(model, file)

## LSTM
---

In [ ]:
model = keras.Sequential([
    layers.Embedding(vocab_size, 128),
    layers.LSTM(128),  # LSTM layer
    layers.Dense(64, activation='relu'),
    layers.Dense(5, activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
y_train_categorical = tf.keras.utils.to_categorical(y_train, num_classes=5)
y_test_categorical = tf.keras.utils.to_categorical(y_test, num_classes=5)
model.fit(x_train, y_train_categorical, epochs=10, batch_size=32)
loss, accuracy = model.evaluate(x_test, y_test_categorical)

print("Loss:", loss)
print("Accuracy:", accuracy)

Epoch 1/10
1611/1611 [==============================] - 17s 8ms/step - loss: 1.0227 - accuracy: 0.5968
Epoch 2/10
1611/1611 [==============================] - 15s 9ms/step - loss: 0.8534 - accuracy: 0.6657
Epoch 3/10
1611/1611 [==============================] - 16s 10ms/step - loss: 0.6983 - accuracy: 0.7381
Epoch 4/10
1611/1611 [==============================] - 25s 15ms/step - loss: 0.5323 - accuracy: 0.8101
Epoch 5/10
1611/1611 [==============================] - 13s 8ms/step - loss: 0.3877 - accuracy: 0.8717
Epoch 6/10
1611/1611 [==============================] - 12s 8ms/step - loss: 0.2908 - accuracy: 0.9071
Epoch 7/10
1611/1611 [==============================] - 12s 8ms/step - loss: 0.2212 - accuracy: 0.9302
Epoch 8/10
1611/1611 [==============================] - 13s 8ms/step - loss: 0.1808 - accuracy: 0.9454
Epoch 9/10
1611/1611 [==============================] - 13s 8ms/step - loss: 0.1468 - accuracy: 0.9555
Epoch 10/10
403/403 [==============================] - 2s 5ms/step - lo

In [ ]:
from sklearn.metrics import classification_report
y_pred_prob = model.predict(x_test)
y_pred = np.argmax(y_pred_prob, axis=1)
print(classification_report(y_test, y_pred))

403/403 [==============================] - 1s 3ms/step
              precision    recall  f1-score   support

         0.0       0.95      0.91      0.93       275
         1.0       0.93      0.94      0.94      1167
         2.0       0.97      0.96      0.96      2043
         3.0       0.95      0.92      0.94      3074
         4.0       0.96      0.98      0.97      6323

    accuracy                           0.96     12882
   macro avg       0.95      0.94      0.95     12882
weighted avg       0.96      0.96      0.96     12882



In [ ]:
import pickle
file_path = '/content/drive/MyDrive/Kaggle/lstm_model.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(model, file)